In [1]:
# TAKEN FROM ORIGINAL AUTHOR: github.com/shnako

# We first parse the input into a list of elements containing [, ] and numbers - no commas as they're unnecessary.
# We then reduce the list of elements by iteratively traversing it and applying the relevant operations.
# The result for the first part is found by recursively calculating the magnitude of the summed lines.
# The result for the second part is found by recursively calculating the maximum magnitude of all pairs of lines. 

In [2]:
from math import floor, ceil
from copy import deepcopy

with open('day_18.txt') as f:
    lines = f.read().splitlines()

In [3]:
def parse_line(line):
    #takes each line and separates it character by character
    elements = filter(lambda element: element != ',', line)
    elements = list(map(lambda element: int(element) if element.isdigit() else element, elements))
    return elements

def reduce_elements(elements):
    has_reduced = True
    while has_reduced:
        has_reduced, elements = try_to_explode(elements)
        if not has_reduced:
            has_reduced, elements = try_to_split(elements)
    return elements

def try_to_explode(elements):
    open_brackets = 0
    for i in range(len(elements)):
        if open_brackets == 4 and elements[i] == '[':
            elements = explode(elements, i)
            return True, elements
        elif elements[i] == '[':
            open_brackets += 1
        elif elements[i] == ']':
            open_brackets -= 1
    return False, elements

def explode(elements, opening_bracket_index):
    left_number = elements[opening_bracket_index + 1]
    right_number = elements[opening_bracket_index + 2]

    for i in range(opening_bracket_index - 1, 0, -1):
        if isinstance(elements[i], int):
            elements[i] += left_number
            break

    for i in range(opening_bracket_index + 4, len(elements)):
        if isinstance(elements[i], int):
            elements[i] += right_number
            break
            
    return elements[:opening_bracket_index] + [0] + elements[opening_bracket_index + 4:]

def try_to_split(elements):
    for i in range(len(elements)):
        if isinstance(elements[i], int) and elements[i] >= 10:
            elements[i:i + 1] = ['[', floor(elements[i] / 2), ceil(elements[i] / 2), ']']
            return True, elements
    return False, elements

def calculate_magnitude(elements, index=0):
    if isinstance(elements[index], int):
        return elements[index], index

    left_sum, index = calculate_magnitude(elements, index + 1)
    right_sum, index = calculate_magnitude(elements, index + 1)
    return 3 * left_sum + 2 * right_sum, index + 1

In [4]:
# part 1
elements = list(map(parse_line, lines))
result = elements[0]
for i in range(1, len(elements)):
    result = ['['] + result + elements[i] + [']']
    result = reduce_elements(result)
calculate_magnitude(result)[0]

4202

In [5]:
# elements = list(map(parse_line, lines))
max_magnitude = 0
for element1 in elements:
    for element2 in elements:
        if element1 == element2:
            continue
        added_element = ['['] + deepcopy(element1) + deepcopy(element2) + [']']
        added_element = reduce_elements(added_element)
        magnitude = calculate_magnitude(added_element)[0]
        if magnitude > max_magnitude:
            max_magnitude = magnitude
max_magnitude

4779